In [0]:
DEBUG = False

CUT_NB = False
KEEP_BLUE = False

GALAXY = 'booi'
GALAXYFULLNAME = 'bootes'

DATA_DIR = '/raid/pfs'

HSC_FILE = f'{DATA_DIR}/data/targeting/dSph/{GALAXYFULLNAME}/{GALAXYFULLNAME}_tpalle_g24.cat'
PMAP_PATH = f'{DATA_DIR}/data/targeting/dSph/{GALAXYFULLNAME}'
GAIA_FILE = f'{DATA_DIR}/data/targeting/dSph/gaia.h5'

ISOCHRONES_PATH = f'{DATA_DIR}/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'

OUTPUT_PATH = f'{DATA_DIR}/data/targeting/dSph/{GALAXYFULLNAME}/'

GAIA_CROSSMATCH_RADIUS = 0.1    # in arcsec

# Prepare the target lists

Pre-process the target list before running netflow. The steps are

* apply color and magnitude cuts
* cross-match with GAIA to include parallaxes and proper motions
* calculate priorities based on a pmap
* calculate the necessary exposure times
* save the pre-processed target list in a uniform format

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

## Load isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))

In [0]:
iso.values.keys()

## Load probability map

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling

In [0]:
fn = 'pmap'
if CUT_NB:
    fn += '_nb'
if KEEP_BLUE:
    fn += '_blue'
fn += '.h5'

pmap = ProbabilityMap(cmd.axes)
pmap.load(os.path.join(PMAP_PATH, fn))

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

l0 = cmd.plot_probability_map(axs[0], pmap, 0)
axs[0].set_title("non-member")

l1 = cmd.plot_probability_map(axs[1], pmap, 1)
axs[0].set_title("member")

f.tight_layout()

In [0]:
# Define a cut based on probability

probcut = ProbabilityCut(pmap, 1, np.log(0.001))

## Load observations

In [0]:
from pfs.ga.targeting.io import ObservationSerializer
from pfs.ga.targeting.util.astro import *

In [0]:
obs = SubaruHSC.text_observation_reader(mags=['r', 'g'], ext=['g', 'r']).read(HSC_FILE)
obs.data.shape

In [0]:
obs.data.columns

In [0]:
obs.data

### Plot observations with color cuts

In [0]:
cmap = plt.get_cmap('tab10')

mask = galaxy.get_selection_mask(obs, nb=False)

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

#ax = f.add_subplot(gs[1])
#ccd.plot_observation(ax, obs, c='lightgray')
#ccd.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=fov.projection.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

f.suptitle('Stars selected based on color cuts.')

## Get GAIA data

In [0]:
import h5py
from pfs.ga.targeting.io import GaiaReader

In [0]:
GAIA_FILE

In [0]:
found = False
if os.path.isfile(GAIA_FILE):
    with h5py.File(GAIA_FILE, 'r') as f:
        found = f'obs/{GALAXY}/gaia' in f

if found:
    print('GAIA data file found, read from local.')
    r = ObservationSerializer()
    gaia = r.read(GAIA_FILE, dataset=f'obs/{GALAXY}/gaia')
    gaia.frame = 'icrs'
    gaia.equinox = 'J2015'
    print(gaia.data.shape)
else:
    print('GAIA data file not found, running query against archive.')
    r = GaiaReader()
    gaia = r.cone_search(galaxy.pos, galaxy.rad)
    print(gaia.data.shape)

    w = ObservationSerializer()
    w.write(gaia, GAIA_FILE, dataset=f'obs/{GALAXY}/gaia')

In [0]:
gaia.data.columns

In [0]:
# Cross-match HSC with GAIA
gaia_idx, separation = obs.cross_match(gaia)
print('median separation', np.median(separation.arcsec))
hsc_gaia_mask = (separation.arcsec < GAIA_CROSSMATCH_RADIUS)
print(hsc_gaia_mask.sum())

In [0]:
# Cross-match HSC with GAIA
hsc_idx, separation = gaia.cross_match(obs)
print('median separation', np.median(separation.arcsec))
gaia_hsc_mask = (separation.arcsec < GAIA_CROSSMATCH_RADIUS)
print(gaia_hsc_mask.sum())

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(6, 3), dpi=240)
gs = f.add_gridspec(1, 2, width_ratios=[2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
gaia_cmd.plot_observation(ax, gaia, c='lightgray')
gaia_cmd.plot_observation(ax, gaia, c='r', mask=gaia_hsc_mask, cmap=cmap)

ax = f.add_subplot(gs[1], projection=wcs.wcs)
fov.plot_observation(ax, gaia, c='lightgray')
fov.plot_observation(ax, gaia, c='r', mask=gaia_hsc_mask, cmap=cmap)

f.suptitle('GAIA stars with HSC counterparts within 1"')

In [0]:
cmap = plt.get_cmap('viridis')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

#ax = f.add_subplot(gs[1])
#ccd.plot_observation(ax, obs, c='lightgray')
#ccd.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

f.suptitle('HSC stars with GAIA counterparts within 1"')

## Merge Gaia proper motions into HSC catalog

In [0]:
# Cross-match HSC with GAIA
hsc_gaia_idx, separation = obs.cross_match(gaia)
print('median separation', np.median(separation.arcsec))
hsc_gaia_mask = (separation.arcsec < 1)

hsc_gaia_mask.shape, hsc_gaia_mask.sum()

In [0]:
gaia.data.columns

In [0]:
columns = ['parallax', 'pm', 'pmdec', 'pmra', 'err_parallax', 'err_pmdec', 'err_pmra']
obs.merge(gaia, hsc_gaia_idx, columns=columns, mask=hsc_gaia_mask)

In [0]:
obs.data.columns

## Assign probabilities

In [0]:
# Hard cuts on magnitudes

selection = galaxy.get_selection_mask(obs, nb=CUT_NB, probcut=probcut)
obs.data.shape, selection.size, selection.sum()

In [0]:
# Look up membership probability

galaxy.assign_probabilities(obs, pmap, mask=selection)
(~np.isnan(obs.data['p_member'])).sum()

In [0]:
cmap = plt.get_cmap('viridis')

mask = selection & ~np.isnan(obs.data['p_member'])

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

#ax = f.add_subplot(gs[1])
#ccd.plot_observation(ax, obs, c='lightgray')
#ccd.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
l = fov.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

f.colorbar(l, ax=ax, label='membership probability')

f.suptitle('Stars selected by color cuts, colored by membersphip probability')

In [0]:
hist, bins = np.histogram(obs.data['p_member'][mask])
plt.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')
plt.xlabel('p')
plt.ylabel('frequency')
plt.title('Distribution of membership probability')

## Assign priorities

In [0]:
# Calculate priorities and required exposure time
# This is implemented differently for every galaxy

galaxy.assign_priorities(obs, mask=None)
obs.data['priority'].unique()

In [0]:
selection.shape, obs.data.shape

In [0]:
obs.data['exp_time'][selection]

In [0]:
cmap = plt.get_cmap('tab10')

obs_mask = obs.data['priority'] >= 0

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

#ax = f.add_subplot(gs[1])
#ccd.plot_observation(ax, obs, c='lightgray')
#ccd.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
l = fov.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

f.colorbar(l, ax=ax, label='target priority')

f.suptitle('Priority class')

In [0]:
obs_mask.size, obs_mask.sum()

In [0]:
# Plot distribution of priorities

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount(obs.data['priority'][obs_mask])
ax.bar(np.arange(hist.size - 1), hist[:-1])

# hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
# plt.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Priority class number distribution')
ax.set_xlabel('Priority class')
ax.set_ylabel('Target count')


In [0]:
# Plot distribution of required visits

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount((np.ceil(obs.data['exp_time'][obs_mask & (obs.data['priority'][obs_mask] < 9)] / 1800.0)).astype(int))
ax.bar(np.arange(hist.size), hist)

# hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
# plt.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Distribution of required visit (t_exp = 1800 s)')
ax.set_xlabel('Number of required visits')
ax.set_ylabel('Target count')


# Save the pre-processed target list

In [0]:
# Only save targets with priority >= 0 and valid values from exp_time
mask = (obs.data['priority'] >= 0) & ~np.isnan(obs.data['exp_time'])

In [0]:
obs.data.columns

In [0]:
obs.data[mask]

In [0]:
# Save the catalog with p_member and priority

fn = os.path.join(OUTPUT_PATH, f'{GALAXYFULLNAME}_obs.feather')
print(fn)

s = DataFrameSerializer()
s.write(obs.data[mask], fn)

obs.data[mask].shape